# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [172]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [174]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID               City      Lat       Lng  Max Temp  Humidity  \
0        0             bethel  41.3712  -73.4140     11.46        90   
1        1             saipan  15.1355  145.7010     29.39        79   
2        2     puerto natales -51.7236  -72.4875      1.73        82   
3        3  sao joao da barra -21.6403  -41.0511     23.35        87   
4        4  wailua homesteads  22.0669 -159.3780     23.40        93   

   Cloudiness  Wind Speed Country        Date  
0         100        0.89      US  1730787121  
1          75        5.14      MP  1730787122  
2          17        3.13      CL  1730787123  
3         100        7.08      BR  1730787124  
4          34        2.49      US  1730787125

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [176]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_3 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    color = "City"
)


# Display the map
map_plot_3

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [178]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather = city_data_df.loc[(city_data_df['Cloudiness'] == 0) & (city_data_df['Wind Speed'] < 4.5)
                                        & (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) ] 


# Drop any rows with null values
# YOUR CODE HERE
#ideal_weather = ideal_weather.append(city_data_df.loc[(city_data_df['City_ID'] == 291)])
# Display sample data
ideal_weather

City_ID             City      Lat       Lng  Max Temp  Humidity  \
69        69           mahina -17.5065 -149.4890     26.87        83   
79        79             yazd  31.8972   54.3675     24.06        27   
165      165  remire-montjoly   4.9167  -52.2667     24.02       100   
238      238          birjand  32.8663   59.2211     26.26        17   
285      285             faaa -17.5500 -149.6000     26.98        83   
342      342            khash  28.2211   61.2158     25.67        16   
412      412           ormara  25.2088   64.6357     26.84        74   
514      514         sultanah  24.4926   39.5857     25.25        33   
516      516           arauca   7.0847  -70.7591     22.98       100   
555      555            pasni  25.2631   63.4710     26.99        75   

     Cloudiness  Wind Speed Country        Date  
69            0        2.06      PF  1730787205  
79            0        3.09      IR  1730787216  
165           0        1.54      GF  1730787325  
238           0        3.09      IR  1730787413  
285           0        2.06      PF  1730787469  
342           0        1.45      IR  1730787537  
412           0        2.36      PK  1730787623  
514           0        2.57      SA  1730787743  
516           0        1.16      CO  1730787745  
555           0        2.65      PK  1730787794

### Step 3: Create a new DataFrame called `hotel_df`.

In [180]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_weather[['City_ID','City','Country','Lat','Lng','Humidity']].copy()

hotel_df['Hotel Name'] = ''
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
hotel_df

City_ID             City Country      Lat       Lng  Humidity Hotel Name
69        69           mahina      PF -17.5065 -149.4890        83           
79        79             yazd      IR  31.8972   54.3675        27           
165      165  remire-montjoly      GF   4.9167  -52.2667       100           
238      238          birjand      IR  32.8663   59.2211        17           
285      285             faaa      PF -17.5500 -149.6000        83           
342      342            khash      IR  28.2211   61.2158        16           
412      412           ormara      PK  25.2088   64.6357        74           
514      514         sultanah      SA  24.4926   39.5857        33           
516      516           arauca      CO   7.0847  -70.7591       100           
555      555            pasni      PK  25.2631   63.4710        75

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [182]:
# Set parameters to search for a hotel
radius = 10000
filters = f"circle:{''},{''},{''}"
bias = f"proximity:{''},{''}"
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =filters
    
    params["bias"] = bias

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mahina - nearest hotel: Motu Martin
yazd - nearest hotel: هتل سنتی مهر
remire-montjoly - nearest hotel: Complexe Belova
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
faaa - nearest hotel: Ancien Hôtel Hilton
khash - nearest hotel: No hotel found
ormara - nearest hotel: No hotel found
sultanah - nearest hotel: فندق جلنار
arauca - nearest hotel: Hotel Nova Park
pasni - nearest hotel: Juddi Hotel


City_ID             City Country      Lat       Lng  Humidity  \
69        69           mahina      PF -17.5065 -149.4890        83   
79        79             yazd      IR  31.8972   54.3675        27   
165      165  remire-montjoly      GF   4.9167  -52.2667       100   
238      238          birjand      IR  32.8663   59.2211        17   
285      285             faaa      PF -17.5500 -149.6000        83   
342      342            khash      IR  28.2211   61.2158        16   
412      412           ormara      PK  25.2088   64.6357        74   
514      514         sultanah      SA  24.4926   39.5857        33   
516      516           arauca      CO   7.0847  -70.7591       100   
555      555            pasni      PK  25.2631   63.4710        75   

                    Hotel Name  
69                 Motu Martin  
79                هتل سنتی مهر  
165            Complexe Belova  
238  مهمانسرای جهانگردی بیرجند  
285        Ancien Hôtel Hilton  
342             No hotel found  
412             No hotel found  
514                 فندق جلنار  
516            Hotel Nova Park  
555                Juddi Hotel

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [184]:
%%capture --no-display

# Configure the map plot
map_plot_4 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    scale = 0.5,
    color = "City",
    frame_width = 800,
    frame_height = 600,  
    hover_cols=['Hotel Name', 'Country']
    
)


# Display the map
map_plot_4

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)